In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline
import dtale

In [2]:
houseprice = pd.read_csv('train.csv', index_col = 0)

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

In [ ]:
houseprice.columns

In [ ]:
houseprice['Total_size']= houseprice['BsmtFinSF1']+ houseprice['BsmtFinSF2'] + houseprice['1stFlrSF'] +houseprice['2ndFlrSF']+houseprice['GarageArea']
houseprice["Total_size"]

In [ ]:
def recent_constr(x):
    if(x.YearBuilt < x.YearRemodAdd):
        return x.YearRemodAdd
    else: 
        return x.YearBuilt

#defining intermediate column 2    
houseprice['Recent_yr'] = houseprice.apply(recent_constr,axis=1)

#time to sale helper defined:
def time_to_sale(x):
    return x.YrSold - x.Recent_yr

#new column defined to be used in Train
houseprice['Age_at_sale']=houseprice.apply(time_to_sale,axis=1)

houseprice["Age_at_sale"]

In [ ]:
houseprice['Outdoor_Porch_Size']= houseprice['WoodDeckSF']+houseprice['OpenPorchSF']
houseprice.reset_index()

#Removing the outlier
houseprice.drop([523,1298],axis=0,inplace=True)

In [ ]:
model2=houseprice[["Neighborhood", "OverallQual", "Total_size", "KitchenQual", "Age_at_sale", "ExterQual", "YrSold",  "LotArea", "BldgType", 'Outdoor_Porch_Size', "Fireplaces", "SalePrice"]]

In [ ]:
model2.dtypes

In [ ]:
model2["SalePrice_log"] = np.log(model2["SalePrice"])

In [ ]:
#Dummifying Neighborhood, ExterQual, KitchenQual, and BldgType columns

from sklearn.preprocessing import OneHotEncoder

#dummify Neighborhood column
Neighborhood_ohe = OneHotEncoder( drop='first', sparse = False )
Neighborhood_enc = Neighborhood_ohe.fit_transform( model2[['Neighborhood']] )
Neighborhood_enc = pd.DataFrame( Neighborhood_enc, columns=Neighborhood_ohe.get_feature_names(['Neighborhood']) )
model2 = pd.concat( (model2.drop(["Neighborhood"], axis=1).reset_index(drop = True), Neighborhood_enc), axis = 1 )

#dummify ExterQual Column
ExterQual_ohe = OneHotEncoder( drop='first', sparse = False )
ExterQual_enc = ExterQual_ohe.fit_transform( model2[['ExterQual']] )
ExterQual_enc = pd.DataFrame( ExterQual_enc, columns=ExterQual_ohe.get_feature_names(['ExterQual']) )
model2 = pd.concat( (model2.drop(["ExterQual"], axis=1).reset_index(drop = True), ExterQual_enc), axis = 1 )


# dummify KitchenQual column
from sklearn.preprocessing import OneHotEncoder
KitchenQual_ohe = OneHotEncoder( drop='first', sparse = False )
KitchenQual_enc = KitchenQual_ohe.fit_transform( model2[['KitchenQual']] )
KitchenQual_enc = pd.DataFrame( KitchenQual_enc, columns=KitchenQual_ohe.get_feature_names(['KitchenQual']) )
model2 = pd.concat( (model2.drop(["KitchenQual"], axis=1).reset_index(drop = True), KitchenQual_enc), axis = 1 )

# dummify BldgType column
from sklearn.preprocessing import OneHotEncoder
BldgType_ohe = OneHotEncoder( drop='first', sparse = False )
BldgType_enc = BldgType_ohe.fit_transform( model2[['BldgType']] )
BldgType_enc = pd.DataFrame( BldgType_enc, columns=BldgType_ohe.get_feature_names(['BldgType']) )
model2 = pd.concat( (model2.drop(["BldgType"], axis=1).reset_index(drop = True), BldgType_enc), axis = 1 )

model2.head()

In [ ]:
y = model2[["SalePrice_log"]]
X = model2.drop(["SalePrice", "SalePrice_log"], axis=1)
print(X.shape)
print(y.shape)

In [ ]:
np.random.seed(0)
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split( X,y, test_size = 0.2  )

In [ ]:
from sklearn import tree
tree_model = tree.DecisionTreeRegressor()

In [ ]:
from sklearn import ensemble

randomForest = ensemble.RandomForestRegressor()
bagging      = ensemble.BaggingRegressor()

In [ ]:
rf_random = RandomizedSearchCV(estimator = randomForest, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# # Fit the random search model
rf_random.fit(Xtrain, ytrain)


In [ ]:
#Random Forest

randomForest.set_params(random_state=42, n_estimators=100, max_features=2)
randomForest.fit(Xtrain, ytrain) # fit 
randomForest.score(Xtrain, ytrain) # accuracy

In [ ]:
print("The training error of random forest is: %.5f" %(1 - randomForest.score(Xtrain, ytrain)))
print("The test     error of random forest is: %.5f" %(1 - randomForest.score(Xtest, ytest)))

In [ ]:
randomForest.score #parameters used for the random forest

In [ ]:
from sklearn.metrics import mean_squared_error
print("Test error is : {}".format(1- randomForest.score(Xtest, ytest)))
mse=(mean_squared_error(ytest, randomForest.predict(Xtest)))
print("Mean Squared Error: {}".format(mse))
rmse = np.sqrt(mean_squared_error(ytest, randomForest.predict(Xtest)))
print("Root Mean Squared Error: {}".format(rmse))

In [ ]:
randomForest.oob_score_

In [ ]:
randomForest.estimators_ #Showing all the trees

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
y_predicted = randomForest.predict(Xtest)

In [ ]:
from sklearn.model_selection import cross_val_predict

fig, ax = plt.subplots(figsize=(16, 8))
ax.scatter(ytest, y_predicted, edgecolors=(1, 1, 1))
ax.plot([ytest.min(), ytest.max()], [ytest.min(), ytest.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title("Actual vs Predicted")

plt.savefig('actual_vs_predicted.png')

In [ ]:
#Plot feature by importance

feature_importance = list(zip(Xtrain.columns, randomForest.feature_importances_))
dtype = [('feature', 'object'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
name, score = zip(*list(feature_sort))

importance = pd.DataFrame({'name':name,'score':score})[:20]
importance

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.barh(importance.name, importance.score, color=(0.2, 0.6, 0.8, 0.6))
plt.gca().invert_yaxis()
ax.set_title('Top 20 Features by Importance')
plt.ylabel('', fontsize=16)

plt.savefig('importance.png')

In [ ]:
n_trees_range = range(30, 500, 10)  # OOB score will warning if too few trees
train_error2 = []
test_error2 = []
oob_error = []

for n_trees in n_trees_range:
    randomForest.set_params(n_estimators=n_trees, random_state=42, oob_score=True)
    randomForest.fit(Xtrain, ytrain)
    train_error2.append(1 - randomForest.score(Xtrain, ytrain))
    test_error2.append(1 - randomForest.score(Xtest, ytest))
    oob_error.append(1 - randomForest.oob_score_)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 5))
plt.plot(n_trees_range, train_error2, c='red', label='training error')
plt.plot(n_trees_range, test_error2, c='blue', label='test error')
plt.plot(n_trees_range, oob_error, c='pink', label='oob error')
plt.ylabel('Error')
plt.xlabel('Number of trees')
plt.legend()

plt.savefig('number of trees.png')

In [ ]:
Xtrain.columns

In [ ]:
# grid_search_forest.best_estimator

In [ ]:
# from sklearn.tree import export_graphviz
# # Export as dot file
# export_graphviz(randomForest_model, out_file='tree.dot', 
#                 #feature_names = Xtrain.columns,
#                # class_names = ytrain.columns,
#                 rounded = True, proportion = False, 
#                 precision = 2, filled = True)

# # Convert to png using system command (requires Graphviz)
# from subprocess import call
# call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# # Display in jupyter notebook
# from IPython.display import Image
# Image(filename = 'tree.png')

In [ ]:
#VIF #AIC, BIC

#Sales price boxplit by neighborhood and KitchenQual

In [ ]:
import statsmodels.api as sm 
X_add_const = sm.add_constant(Xtrain)
ols = sm.OLS(ytrain, X_add_const)
ans = ols.fit()
print(ans.summary())


In [ ]:
ans.summary().tables[1]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

N = add_constant(Xtrain)
VIF= pd.DataFrame([variance_inflation_factor(N.values, i) 
               for i in range(N.shape[1])], index=N.columns)

#VIF.reset_index()
VIF = VIF.rename(columns={'0': 'VIF'}, index={'index': 'Name'})

VIF.reset_index()
#VIF.columns
#VIF.sort_values(by="index[0]")

In [ ]:
#GridSearch to find the best score

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = [3]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 3]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available scores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(Xtrain, ytrain)


In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(Xtrain, ytrain)
base_accuracy = evaluate(base_model, Xtest, ytest)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, Xtest, ytest)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import ensemble

randomForest = ensemble.RandomForestRegressor()
randomForest.set_params(random_state=42, n_estimators=100, max_features=2)
randomForest.fit(Xtrain, ytrain) # fit 
randomForest.score(Xtest, ytest) # accuracy